# Input Description

Job  Machine
3    5
Job# Machine#   M1     M2     M3            VG
1    3          1  10  2  30  3  20         1
1    4          1  12  5  13  3  21  4  28  2
1    3          1  21  2  10  3  11         3

3 5
1 3 1 10 2 30 3 20 1
1 4 1 12 5 13 3 21 4 28 2
1 3 1 21 2 10 3 11 3

# Data Parsing

In [1]:
def readFile(filePath):
    with open(filePath) as fp:
        line = fp.readline()
        nm = line.strip().split()
        machineCnt = nm[1]
        jobs = []
        machines = []
        processing_time = []
        VG = []

        for i in range(int(nm[0])):
            line = fp.readline()
            cv = list(map(int, line.split()))
            operations = []
            machine_l = []
            times = []
            j = 1
            while j < len(cv):
                # 각 op 별 machine, processing_time 할당
                k = cv[j]
                operation = []
                for kj in range(k):
                    j = j + 1
                    if (j >= len(cv)):
                        break;
                    # machine 할당
                    machine = cv[j]
                    j = j + 1
                    # processing_time 할당
                    processing_t = cv[j]
                    machine_l.append(machine)
                    processing_time.append(processing_t)
                    operation.append({'machine': machine, 'processingTime': processing_t})
                j = j + 1
                operations.append(operation)
            machines.append(machine_l)

            # VG 할당
            if (j > len(cv)):
                VG.append(int(cv[-1]))

            jobs.append(operations)
    return jobs, VG, machines, processing_time, {'machineCnt': int(machineCnt), 'jobs': jobs}


In [2]:
jobs, VG, machines, processing_time, jobDicts = readFile("./test_data.txt")

# Population Generation

In [3]:
import random

def generateN(jobDicts, VG):
    N = []
    job_n = 0
    if (len(VG) > 0):    
        for v in VG:
            for i in range(v):
                N.append(job_n)
            job_n += 1

        random.shuffle(N)
    else:
        jobs = jobDicts['jobs']
        i = 0
        for job in jobs:
            for op in job:
                N.append(i)
            i = i+1

        random.shuffle(N)
    return N

In [4]:
def generateM(jobDicts, VG):
    M = []
    if(len(VG) > 0):
        cnt = 0
        for v in VG:
            for i in range(v):
                random.shuffle(machines[cnt])
                M.append(machines[cnt][i])
            cnt += 1
    else:
        jobs = jobDicts['jobs']
        cnt = 0
        for job in jobs:
            for op in job:
                cnt += 1
                randomMachine = random.randint(0, len(op)-1)
                M.append(randomMachine)
    return M

In [5]:
def initial_generation(jobDicts, VG):
    pop_gen = []
    for i in range(10):
        NS = generateN(jobDicts, VG)
        MS = generateM(jobDicts, VG)
        pop_gen.append((NS, MS))

    return pop_gen

In [6]:
parameter = initial_generation(jobDicts,VG) # NS MS
parameter

[([0, 2, 3, 3, 1, 3, 4, 4, 3, 4, 2, 0, 1, 3, 0, 2, 2, 2],
  [0, 3, 1, 0, 3, 0, 0, 0, 2, 0, 1, 1, 1, 1, 0, 1, 0, 1]),
 ([3, 3, 2, 4, 2, 0, 0, 3, 4, 2, 4, 2, 0, 3, 1, 1, 2, 3],
  [0, 1, 2, 2, 3, 1, 1, 0, 1, 1, 0, 3, 1, 3, 1, 2, 1, 1]),
 ([3, 2, 4, 0, 0, 4, 2, 0, 3, 4, 2, 3, 1, 2, 1, 3, 2, 3],
  [0, 1, 3, 3, 0, 0, 2, 0, 1, 0, 0, 1, 0, 2, 1, 1, 2, 0]),
 ([4, 2, 2, 0, 4, 4, 1, 0, 3, 3, 0, 3, 3, 2, 2, 2, 1, 3],
  [0, 1, 3, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1]),
 ([3, 1, 0, 4, 3, 2, 2, 2, 0, 4, 3, 3, 3, 2, 1, 4, 0, 2],
  [1, 1, 1, 0, 1, 1, 2, 2, 3, 0, 1, 2, 0, 3, 0, 0, 1, 0]),
 ([0, 1, 4, 4, 2, 3, 2, 2, 3, 0, 3, 1, 2, 3, 2, 0, 3, 4],
  [0, 2, 1, 2, 3, 0, 1, 1, 1, 0, 2, 1, 0, 2, 1, 1, 2, 0]),
 ([0, 4, 2, 2, 0, 2, 2, 3, 2, 3, 4, 3, 0, 3, 1, 3, 1, 4],
  [0, 2, 2, 3, 3, 1, 2, 0, 1, 0, 1, 2, 0, 2, 1, 1, 0, 1]),
 ([3, 1, 0, 0, 4, 2, 3, 4, 3, 1, 3, 2, 2, 2, 2, 3, 4, 0],
  [1, 0, 3, 2, 3, 1, 2, 0, 3, 1, 1, 0, 0, 0, 0, 2, 1, 1]),
 ([0, 1, 2, 3, 0, 4, 3, 4, 2, 4, 2, 2, 3, 3, 1, 3, 0, 2],
  [1, 

# Calculate Makespan

In [7]:
# VG 별 machine 나누기
def split_by_machine(VG, param, jobDicts):
    (n,m) = param
    job_lists = []
    current = 0
    if(len(VG) > 0):
        for v in VG:
            append_l = []
            for i in range(v):
                append_l.append(m[current])
                current += 1
            job_lists.append(append_l)
    else:
        current = 0
        for index, job in enumerate(jobDicts['jobs']):
            job_lists.append(m[current:current+len(job)])
            current += len(job)

    return job_lists

In [8]:
def is_free(tab, start, duration):
    for k in range(start, start+duration):
        if not tab[k]:
            return False
    return True

def find_first_proc_time(start_ctr, duration, machine_jobs):
    max_duration_list = []
    max_duration = start_ctr + duration

    # max_duration is either the start_ctr + duration or the max(possible starts) + duration
    if machine_jobs:
        for job in machine_jobs:
            max_duration_list.append(job[2] + job[1])  # start + process time

        max_duration = max(max(max_duration_list), start_ctr) + duration

    machine_used = [True] * max_duration

    # Updating array with used places
    for job in machine_jobs:
        start = job[2]
        long = job[1]
        for k in range(start, start + long):
            machine_used[k] = False

    # Find the first available place that meets constraint
    for k in range(start_ctr, len(machine_used)):
        if is_free(machine_used, k, duration):
            return k

In [9]:
def calculateMakespan(mo):
    # Getting the max for each machine
    max_per_machine = []
    for machine in mo:
        max_d = 0
        for job in machine:
            end = job[2]# job[2] + job[1]
            if end > max_d:
                max_d = end
        max_per_machine.append(max_d)
    
    return max(max_per_machine)

In [10]:
def getMakespans(cpl, jobDicts):
    o = jobDicts['jobs']
    (n, m) = cpl

    cnt = 0
    ops = [[] for i in range(jobDicts['machineCnt'])]
    time_taken = [[] for i in range(jobDicts['machineCnt'])]
    m_splits = split_by_machine(VG, (n, m), jobDicts)

    outer_shell = [0] * len(m_splits)
    time_outer_shell = [0] * len(m_splits)

    for i in n:
        idx_m = m_splits[i][outer_shell[i]]
        idx_m = idx_m-1

        if (len(VG) > 0):
            machine = [i['machine'] for i in o[i][0] if i['machine']==idx_m+1][0]
            proc_t = [i['processingTime'] for i in o[i][0] if i['machine']==idx_m+1][0]
        else: 
            machine = o[i][outer_shell[i]][idx_m]['machine']
            proc_t = o[i][outer_shell[i]][idx_m]['processingTime']
        start_t = time_outer_shell[i]

        outer_shell[i] += 1    
        tag = "{}-{}".format(i, outer_shell[i]+1)
        if(len(time_taken[machine-1])>0):
            time = (start_t + proc_t)
            lng = len(time_taken[machine-1])-1
            time = time_taken[machine-1][lng] + time
            time_taken[machine-1].append(time)
            ops[machine-1].append((tag, time_taken[machine-1][lng], time))
        else:
            time_taken[machine-1].append((start_t + proc_t))
            ops[machine-1].append((tag, start_t, (start_t + proc_t)))
    
    calc = calculateMakespan(ops)
    print(calc)
    return ops




In [11]:
sortedPop = sorted(parameter, key=lambda cpl: getMakespans(cpl, jobDicts))

136
188
183
128
150
108
111
176
112
172


def getMakespans(cpl, jobDicts):
    o = jobDicts['jobs']
    (n, m) = cpl

    cnt = 0
    ops = [[] for i in range(jobDicts['machineCnt'])]
    m_splits = split_by_machine(VG, (n, m), jobDicts)

    outer_shell = [0] * len(m_splits)
    time_outer_shell = [0] * len(m_splits)

    for i in n:
        idx_m = m_splits[i][outer_shell[i]]
        idx_m = idx_m-1

        machine = o[i][outer_shell[i]][idx_m]['machine']
        proc_t = o[i][outer_shell[i]][idx_m]['processingTime']
        start_t = time_outer_shell[i]
        
        if(len(ops[i]) == 0):
            start = 0
        else:
            start = find_first_proc_time(start_t, proc_t, ops[machine-1])
        tag = "{}-{}".format(i, machine)
        # print(start, start_t, proc_t, machine)
        ops[machine-1].append((tag, proc_t, start_t, machine))

        outer_shell[i] += 1
        time_outer_shell[i] = (start + proc_t)
        
        

    mo = ops
    calc = calculateMakespan(mo)
    print(calc, ops)
    return ops
    